In [1]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
import random
from nlpaug.util import Action


2024-05-15 16:36:17.750239: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-15 16:36:17.786209: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 16:36:17.786244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 16:36:17.787372: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-15 16:36:17.793907: I tensorflow/core/platform/cpu_feature_guar

### Load the original dataset

In [15]:
import pandas as pd
language = 'fr'
from datasets import load_dataset
# Load the WikiANN dataset
ds_orig = load_dataset("wikiann", language)
df_test = pd.DataFrame(ds_orig['train'])

In [27]:
df_test

,tokens,ner_tags,langs,spans
0,"[Shortly, afterward, ,, an, encouraging, respo...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, ...","[en, en, en, en, en, en, en, en, en, en, en, e...","[LOC: India, LOC: Adyar]"
1,"[:, Kanye, West, featurjng, Jamie, Foxx, —, ``...","[0, 1, 2, 0, 1, 2, 0, 0, 3, 4, 0, 0, 0, 0]","[en, en, en, en, en, en, en, en, en, en, en, e...","[PER: Kanye West, PER: Jamie Foxx, ORG: Gold D..."
2,"[Blacktown, raiiway, station]","[3, 4, 4]","[en, en, en]",[ORG: Blacktown railway station]
3,"['', Mycalesis, perseus, lalassis, '', (, Hwwi...","[0, 5, 6, 6, 0, 0, 0, 0, 0, 0]","[en, en, en, en, en, en, en, en, en, en]",[LOC: Mycalesis perseus lalassis]
4,"[Jomny, Lee, Miller, -, Eli, Stone, '']","[1, 2, 2, 0, 3, 4, 0]","[en, en, en, en, en, en, en]","[PER: Jonny Lee Miller, ORG: Eli Stone]"
...,...,...,...,...
9995,"[Tonj, Stewart, ', '', (, PC4, ), ', '']","[1, 2, 0, 0, 0, 0, 0, 0, 0]","[en, en, en, en, en, en, en, en, en]",[PER: Tony Stewart]
9996,"[Maryland, Rojte, 472]","[3, 4, 4]","[en, en, en]",[ORG: Maryland Route 472]
9997,"[Rentin, ,, Washington]","[5, 6, 6]","[en, en, en]","[LOC: Renton , Washington]"
9998,"[He, served, as, a, member, of, the, South, Ea...","[0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 0]","[en, en, en, en, en, en, en, en, en, en, en]",[ORG: South Eastern Circuit]


In [45]:
import os
print(os.environ.get('CUDA_PATH'))

None


### Keyboard Augmentation [ https://nlpaug.readthedocs.io/en/latest/augmenter/char/keyboard.html ]
percent_to_replace=20
max_aug = 4
min_aug = 1

In [16]:
import pandas as pd
aug = nac.KeyboardAug(aug_char_max=1 ,aug_word_max=4,aug_word_p=0.2, include_special_char=False,include_upper_case =False,include_numeric=False, lang=language)


In [17]:
import copy
df2 = df_test.copy(deep=True)

# Function to replace a percentage of tokens in a sentence
def replace_noisy(tokens, percent_to_replace=30 , max_aug=10, min_aug=1):
    
    word_indices = [index for index, token in enumerate(tokens) if token.isalpha()]
#     print(len(word_indices))
    if len(word_indices) ==0:
        return tokens
    percent_replace = int(len(word_indices) * (percent_to_replace / 100))
#     index_list = random.sample(range(0, len(tokens)),len(tokens)-1)
    if percent_replace >= max_aug:
        num_tokens_to_replace= max_aug
    elif percent_replace < min_aug:
        num_tokens_to_replace = min_aug
    else: num_tokens_to_replace = percent_replace
    tokens_copy = copy.deepcopy(tokens)
    for i in range(num_tokens_to_replace):
        rep_i=random.sample(word_indices,1)[0]
        tokens_copy[rep_i] = aug.augment(tokens_copy[rep_i])[0]
    
    return tokens_copy

# Applying the custom function to the 'sentence' column with a specified percentage (e.g., 30%)
df2['tokens'] = df2['tokens'].apply(lambda x: replace_noisy(x, percent_to_replace=20,max_aug = 4,min_aug = 1))


In [18]:

print(' '.join(df2.tokens[0]))
# print(' '.join(df_test.tokens[0]) )   

Saxofrage faux Orpin


In [44]:
print(' '.join(df_test.tokens[0]) )   

Shortly afterward , an encouraging response influenced him to go to India ; he arrived at Adyar in 1884 .


### Save the augmented dataset

In [19]:
from datasets import Dataset
ds_noisy = Dataset.from_pandas(df2 , features = ds_orig['train'].features,preserve_index=False)
ds_noisy.save_to_disk("/wikiann_nlpaug/train/"+language+"_noisy.hf")

Saving the dataset (0/1 shards):   0%|          | 0/20000 [00:00<?, ? examples/s]